# City assistant - Городской ассистент
В этом проекте будет создан ассистент с набором инструментов. В качестве инструментов будут следующие функции:

*   определение погоды;
*   поиск ресторанов;
*   поиск достопримечательностей;
*   поиск новостей.

Для реализации такого агента нам понадобится:

*   gigachain для создания агента;
*   WeatherAPI ключ для погоды;
*   2GIS Places API для поиска ресторанов и достопримечательностей;
*   NewsAPI для поиска новостей.

### Установка зависимостей

In [3]:
!pip install -q langchain-gigachat langgraph langchain-community

### Инициализация GigaChat
Настройка GigaChat, импорт необходимых модулей, загрузка данных из секрета colab


In [26]:
from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from google.colab import userdata
import requests
import json
from datetime import datetime, timedelta

auth = userdata.get('SBER_AUTH')

### Инструмент - Погода
Назначение: Получение текущей погоды и прогноза через WeatherAPI:

*   Запрос к WeatherAPI с параметрами города, дней прогноза
*   Данные: температура, влажность, ветер, УФ-индекс, качество воздуха
*   Читаемый вывод с структурированной информацией

In [27]:
from langchain_core.tools import tool

@tool
def get_weather(city: str, days: int = 3, include_aqi: bool = False, include_alerts: bool = False) -> str:
    """Получает текущий прогноз погоды и прогноз на несколько дней для указанного города через WeatherAPI.

    Args:
        city: Название города или местоположения
        days: Количество дней прогноза (1-14, по умолчанию 3)
        include_aqi: Включать данные о качестве воздуха
        include_alerts: Включать погодные предупреждения
    """
    try:
        api_key = userdata.get('WeatherAPI_key')

        params = {
            'key': api_key,
            'q': city,
            'lang': 'ru'
        }

        if days > 1:
            url = "http://api.weatherapi.com/v1/forecast.json"
            params['days'] = min(days, 14)  # Ограничиваем максимум 14 дней
        else:
            url = "http://api.weatherapi.com/v1/current.json"

        if include_aqi:
            params['aqi'] = 'yes'
        if include_alerts:
            params['alerts'] = 'yes'

        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code == 200:
            return format_weather_response(data, days)
        else:
            error_msg = data.get('error', {}).get('message', 'Неизвестная ошибка')
            return f"Не удалось получить погоду для {city}: {error_msg}"

    except Exception as e:
        return f"Ошибка при получении погоды: {str(e)}"

def format_weather_response(data: dict, days: int) -> str:
    """Форматирует ответ от WeatherAPI в читаемый текст."""

    location = data['location']
    result = f"Погода в {location['name']}, {location['country']}\n"
    result += f"Координаты: {location['lat']}, {location['lon']}\n"
    result += f"Местное время: {location['localtime']}\n\n"

    if 'current' in data:
        current = data['current']
        result += "Текущая погода:\n"
        result += f"• Состояние: {current['condition']['text']}\n"
        result += f"• Температура: {current['temp_c']}°C (ощущается как {current['feelslike_c']}°C)\n"
        result += f"• Влажность: {current['humidity']}%\n"
        result += f"• Ветер: {current['wind_kph']} км/ч, направление: {current['wind_dir']}\n"
        result += f"• Давление: {current['pressure_mb']} мбар\n"
        result += f"• Видимость: {current['vis_km']} км\n"
        result += f"• УФ-индекс: {current.get('uv', 'N/A')}\n"

        if 'air_quality' in current:
            aqi = current['air_quality'].get('us-epa-index')
            if aqi:
                aqi_levels = {1: "Хорошее", 2: "Умеренное", 3: "Нездоровое для чувствительных групп",
                             4: "Нездоровое", 5: "Очень нездоровое", 6: "Опасное"}
                result += f"• Качество воздуха: {aqi_levels.get(aqi, 'N/A')}\n"

    if 'forecast' in data and days > 1:
        result += f"\n Прогноз на {days} дней:\n"
        for day_data in data['forecast']['forecastday']:
            date = day_data['date']
            day = day_data['day']
            astro = day_data['astro']

            result += f"\n {date}:\n"
            result += f"• Макс: {day['maxtemp_c']}°C, Мин: {day['mintemp_c']}°C\n"
            result += f"• Состояние: {day['condition']['text']}\n"
            result += f"• Вероятность дождя: {day.get('daily_chance_of_rain', 0)}%\n"
            result += f"• Вероятность снега: {day.get('daily_chance_of_snow', 0)}%\n"
            result += f"• Восход: {astro['sunrise']}, Закат: {astro['sunset']}\n"

            if date == datetime.now().strftime('%Y-%m-%d') and 'hour' in day_data:
                result += "• Сегодня по часам: "
                hours_info = []
                for hour in day_data['hour'][::3]:  # Каждые 3 часа
                    time = hour['time'].split()[1][:5]
                    temp = hour['temp_c']
                    condition = hour['condition']['text']
                    hours_info.append(f"{time}h: {temp}°C ({condition})")
                result += "; ".join(hours_info[:4]) + "\n"

    if 'alerts' in data and data['alerts'].get('alert'):
        result += "\nПогодные предупреждения:\n"
        for alert in data['alerts']['alert']:
            result += f"• {alert['headline']}\n"
            result += f"  Действует до: {alert['expires']}\n"

    return result

### Инструмент - Достопримечательности
Назначение: Поиск популярных достопримечательностей через 2ГИС API:

*   Запрос к 2ГИС API с городом, где искать достопримечательности;
*   Вывод: Список достопримечательностей с адресами.



In [28]:
@tool
def find_attractions(city: str) -> str:
    """Ищет популярные достопримечательности в указанном городе через 2ГИС Places API."""
    try:
        api_key = userdata.get('2GIS_API')

        geocode_url = f"https://catalog.api.2gis.com/3.0/items/geocode?q={city}&key={api_key}&fields=items.point"
        geocode_response = requests.get(geocode_url)
        geocode_data = geocode_response.json()

        if geocode_data['meta']['code'] == 200 and geocode_data['result']['items']:
            location = geocode_data['result']['items'][0]['point']
            lat, lon = location['lat'], location['lon']

            places_url = f"https://catalog.api.2gis.com/3.0/items?q=достопримечательности&location={lon},{lat}&radius=5000&key={api_key}&fields=items.name,items.address_name&page_size=5"
            places_response = requests.get(places_url)
            places_data = places_response.json()

            if places_data['meta']['code'] == 200 and places_data['result']['items']:
                attractions = []
                for place in places_data['result']['items']:
                    name = place['name']
                    address = place.get('address_name', 'адрес не указан')
                    attractions.append(f"• {name} ({address})")

                return f"Достопримечательности в {city}:\n" + "\n".join(attractions)
            else:
                backup_url = f"https://catalog.api.2gis.com/3.0/items?q=музеи,парки&location={lon},{lat}&radius=5000&key={api_key}&fields=items.name,items.address_name&page_size=5"
                backup_response = requests.get(backup_url)
                backup_data = backup_response.json()

                if backup_data['meta']['code'] == 200 and backup_data['result']['items']:
                    places = []
                    for place in backup_data['result']['items']:
                        name = place['name']
                        address = place.get('address_name', 'адрес не указан')
                        places.append(f"• {name} ({address})")

                    return f"Интересные места в {city}:\n" + "\n".join(places)
                else:
                    return f"Не удалось найти достопримечательности в {city}"
        else:
            return f"Не удалось определить местоположение города {city}"

    except Exception as e:
        return f"Ошибка при поиске достопримечательностей: {str(e)}"

### Инструмент - Рестораны
Назначение: Поиск ресторанов и кафе через 2ГИС API:

*   Запрос к 2ГИС API с городом, где искать рестораны;
*   Вывод: Список ресторанов с адресами.

In [29]:
@tool
def find_restaurants(city: str) -> str:
    """Ищет рестораны, кафе и места для питания в указанном городе через 2ГИС Places API."""
    try:
        api_key = userdata.get('2GIS_API')

        geocode_url = f"https://catalog.api.2gis.com/3.0/items/geocode?q={city}&key={api_key}&fields=items.point"
        geocode_response = requests.get(geocode_url)
        geocode_data = geocode_response.json()

        if geocode_data['meta']['code'] == 200 and geocode_data['result']['items']:
            location = geocode_data['result']['items'][0]['point']
            lat, lon = location['lat'], location['lon']

            restaurants_url = f"https://catalog.api.2gis.com/3.0/items?q=рестораны,кафе&location={lon},{lat}&radius=3000&key={api_key}&fields=items.name,items.address_name&page_size=5"
            restaurants_response = requests.get(restaurants_url)
            restaurants_data = restaurants_response.json()

            if restaurants_data['meta']['code'] == 200 and restaurants_data['result']['items']:
                restaurants = []
                for place in restaurants_data['result']['items']:
                    name = place['name']
                    address = place.get('address_name', 'адрес не указан')
                    restaurants.append(f"• {name} ({address})")

                return f"Рестораны и кафе в {city}:\n" + "\n".join(restaurants)
            else:
                return f"Не удалось найти рестораны в {city}"
        else:
            return f"Не удалось определить местоположение города {city}"

    except Exception as e:
        return f"Ошибка при поиске ресторанов: {str(e)}"


### Инструмент - Новости
Назначение: Поиск актуальных новостей о городе через NewsAPI:

*   Запрос к NewsAPI с городом, где искать новости;
*   Вывод: последние новости.

In [30]:
@tool
def find_news(city: str) -> str:
    """Ищет последние новости о городе через NewsAPI."""
    try:
        api_key = userdata.get('NewsAPI')

        from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

        news_url = f"https://newsapi.org/v2/everything?q={city}&from={from_date}&sortBy=publishedAt&language=ru&pageSize=5&apiKey={api_key}"
        response = requests.get(news_url)
        data = response.json()

        if data['status'] == 'ok' and data['totalResults'] > 0:
            news_items = []
            for article in data['articles'][:5]:
                title = article['title']
                source = article['source']['name']
                published_at = article['publishedAt'][:10]
                description = article.get('description', 'Описание отсутствует')

                if len(description) > 150:
                    description = description[:150] + "..."

                news_items.append(f"• {title} ({source}, {published_at})\n  {description}")

            return f"Последние новости о {city}:\n" + "\n\n".join(news_items)
        else:
            return f"Не удалось найти новости о {city} за последнюю неделю"

    except Exception as e:
        return f"Ошибка при поиске новостей: {str(e)}"

### Создание агента с набором инструментов


In [31]:
travel_tools = [get_weather, find_attractions, find_restaurants, find_news]

model = GigaChat(
    credentials=auth,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-2",
    verify_ssl_certs=False,
    max_tokens=1024
)

agent_executor = create_react_agent(model, travel_tools)


### Интерфейс для общения с агентом
Взаимодействие: Цикл вопрос-ответ с обработкой ввода

Обработка ответов: Фильтрация и форматирование вывода агента

Управление: Возможность выхода командой "выход"

In [33]:
def chat_with_travel_agent():
    """Функция для чата с агентом-планировщиком путешествий"""
    print("Добро пожаловать в планировщика путешествий!")
    print("Задавайте вопросы о любом городе! Например:")
    print("- Какая погода в Волгограде?")
    print("- Какие есть достопремичательности в Казани?")
    print("- Где поесть в Воронеже?")
    print("- Какие новости в Омске?")
    print("- 'выход' (для завершения)")

    while True:
        user_input = input("\nВопрос: ").strip()

        if user_input.lower() in ['выход']:
            print("До свидания!")
            break

        if not user_input:
            continue

        try:

            result = agent_executor.invoke({
                "messages": [HumanMessage(content=user_input)]
            })

            tool_responses = []
            final_ai_response = None

            for message in result["messages"]:
                if hasattr(message, 'content') and message.content == user_input:
                    continue

                if (hasattr(message, 'content') and message.content and
                    message.content != user_input and
                    not hasattr(message, 'tool_calls')):
                    content = message.content
                    if any(indicator in content for indicator in
                           ['Достопримечательности', 'Погода', 'Рестораны', 'новости',
                            'рейтинг', '°C', 'температура', 'влажность']):
                        tool_responses.append(content)
                    elif not final_ai_response:
                        final_ai_response = content

            if tool_responses:
                combined_response = "\n".join(tool_responses)
                if len(combined_response) > 1500:
                    combined_response = combined_response[:1500] + "..."
                print(f"\nОтвет: {combined_response}")
            elif final_ai_response:
                if len(final_ai_response) > 1500:
                    final_ai_response = final_ai_response[:1500] + "..."
                print(f"\nОтвет: {final_ai_response}")
            else:
                print("\nНе удалось получить ответ от агента")

        except Exception as e:
            print(f"Произошла ошибка: {str(e)}")

chat_with_travel_agent()

Добро пожаловать в планировщика путешествий!
Задавайте вопросы о любом городе! Например:
- Какая погода в Волгограде?
- Какие есть достопремичательности в Казани?
- Где поесть в Воронеже?
- Какие новости в Омске?
- 'выход' (для завершения)

Вопрос: Какие есть достопремичательности в Казани?

Ответ: Достопримечательности в Казань:
• Дворец Земледельцев (Федосеевская улица, 36)
• Собор Казанской иконы Божией Матери в Казанско-Богородицком монастыре (Большая Красная улица, 5Б)
• Казанский национальный исследовательский технический университет им. А.Н. Туполева-КАИ, приемная ректора (улица Карла Маркса, 10)
• Ирек, мечеть (Федосеевская улица, 5)
• Казанский Богородицкий мужской монастырь (Большая Красная улица, 5)

Вопрос: Где поесть в Воронеже?

Ответ: Рестораны и кафе в Воронеж:
• Belgium, ресторан (площадь Ленина, 8)
• Мидийное место, кафе (Пушкинская улица, 2)
• Just Bar&Kitchen, кафе-бар (Пушкинская улица, 1)
• ЕваЕла, рестобистро (улица Комиссаржевской, 7)
• Brixton art bar (улица Со